## this is my title
- this is a point
> this is an indent
[ this is a hyperlink](https://www.hcrma.net/)

In [2]:
import numpy as np

In [7]:
# -*- coding: utf-8 -*-

# Simple s2 KML generator
# Copyright 2018, Sean C. Nichols
#  seanni@trichotomy.ca

# Hereby released into the public domain: feel free to abuse this code as you see fit!
#

# usage: python s2gen.py <max_lat> <min_lat> <max_lng> <min_lng> <s2_lvl>
#
# (i.e.: N-S-E-W order)
#

from __future__ import print_function

from sys import argv, stderr

from s2sphere import RegionCoverer, Cell, LatLng, LatLngRect


In [8]:
def usage(msg=None):
    if msg:
        stderr.write(msg + '\n\n')
    stderr.write('usage: {} <max_lat> <min_lat> <max_lng> <min_lng> <s2_lvl>\n'.format(argv[0]))
    exit(1)

In [9]:
def swap_latlong(latlong):
    return '{},{}'.format(*latlong.split(',')[::-1])


In [14]:
def s2gen(max_lat, min_lat, max_lng, min_lng, s2_lvl):
    point_nw = LatLng.from_degrees(max_lat, min_lng)
    point_se = LatLng.from_degrees(min_lat, max_lng)

    rc = RegionCoverer()
    rc.min_level = s2_lvl
    rc.max_level = s2_lvl
    rc.max_cells = 1000000

    cellids = rc.get_covering(LatLngRect.from_point_pair(point_nw, point_se))

    kml = (
        '<?xml version="1.0" encoding="UTF-8"?>'
        '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">'
        '<Document><name>{0}</name>'
        '<Style id="s2_poly_style"><LineStyle><color>ff0000ff</color><width>2</width></LineStyle><PolyStyle><fill>0</fill></PolyStyle></Style>'
        '<Folder><name>{0}</name><open>1</open>'
    ).format('S2 cells level ' + str(s2_lvl))
    for cid in cellids:
        vertices = [LatLng.from_point(Cell(cid).get_vertex(v)) for v in range(4)]
        kml_coords = ['{},0'.format(swap_latlong(str(v).split()[-1])) for v in vertices]
        kml += (
            '<Placemark><name>{}</name><styleUrl>#s2_poly_style</styleUrl><Polygon><tessellate>1</tessellate><outerBoundaryIs><LinearRing>'
            '<coordinates>{} {}</coordinates>'
            '</LinearRing></outerBoundaryIs></Polygon></Placemark>'
        ).format(cid.id(), ' '.join(kml_coords), kml_coords[0])
    kml += (
        '</Folder></Document>'
        '</kml>'
    )

    return kml

In [31]:
max_lat = 26.199979 #Horizontal Point on the West
min_lat = 26.122095 #Horizontal Point on the East
max_lng = -97.957739 #Vertical Point to the North
min_lng = -98.023402 #Vertical Point to the South
s2_lvl = 17 #S2 level, e.g. L17 or L14 as an Integer

### Give file unique name(s) so it doesn't append to existing kml's

In [28]:
%%capture cap --no-stderr
print (s2gen(max_lat, min_lat, max_lng, min_lng, s2_lvl))
with open('s2grid.kml', 'w') as f:
    f.write(cap.stdout)